# ACCESS-MOPPeR Getting Started

## Set up configuration

When you first import `access_mopper` in a Python environment, the package will automatically create a `user.yml` file in your home directory (`~/.mopper/user.yml`).  

During this initial setup, you will be prompted to provide some basic information, including:  
- Your name  
- Your email address  
- Your work organization
- Your ORCID

This information is stored in `user.yml` and will be used as global attributes in the files generated during the CMORisation process. This ensures that each CMORised file includes metadata identifying who performed the CMORisation, allowing us to track data provenance and follow up with the responsible person if needed.

In [1]:
from access_mopper import ACCESS_ESM_CMORiser
import dask.distributed as dask


Loaded Configuration:
Creator Name: Romain Beucher
Organisation: ACCESS-NRI
Creator Email: romain.beucher@anu.edu.au
Creator URL: 0000-0003-3891-5444


In [2]:
client = dask.Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 15.06 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40117,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.06 GiB
Comm: tcp://127.0.0.1:35209,Total threads: 1
Dashboard: http://127.0.0.1:42453/status,Memory: 1.25 GiB
Nanny: tcp://127.0.0.1:41919,


In [3]:
import glob
files = glob.glob("../../Test_data/esm1-6/atmosphere/aiihca.pa-0961*_mon.nc")

In [4]:
parent_experiment_config = {
        "parent_experiment_id": "piControl",
        "parent_activity_id": "CMIP",
        "parent_source_id": "ACCESS-ESM1-5",
        "parent_variant_label": "r1i1p1f1",
        "parent_time_units": "days since 0001-01-01 00:00:00",
        "parent_mip_era": "CMIP6",
        "branch_time_in_child": 0.0,
        "branch_time_in_parent": 54786.0,
        "branch_method": "standard"
}

In [5]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=files,
    compound_name="Amon.rsds",
    experiment_id="historical",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    parent_info=parent_experiment_config)

In [6]:
cmoriser.run()

In [7]:
ds = cmoriser.to_dataset()

In [8]:
ds

<xarray.Dataset> Size: 1MB
Dimensions:    (lat: 145, time: 12, bnds: 2, lon: 192)
Coordinates:
  * lat        (lat) float64 1kB -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
  * lon        (lon) float64 2kB 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
  * time       (time) float64 96B 3.509e+05 3.507e+05 ... 3.51e+05 3.507e+05
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 28kB dask.array<chunksize=(1, 145, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 37kB dask.array<chunksize=(1, 192, 2), meta=np.ndarray>
    time_bnds  (time, bnds) float64 192B dask.array<chunksize=(1, 2), meta=np.ndarray>
    rsds       (lon, lat, time) float32 1MB dask.array<chunksize=(192, 145, 1), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    creation_date:          2025-07-30T03:13:29Z
    data_specs_version:     01.00.33
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    ...                     ...
    branch_method:          standard
    external_variables:     areacella
    creator_name:           Romain Beucher
    creator_organisation:   ACCESS-NRI
    creator_email:          romain.beucher@anu.edu.au
    creator_url:            0000-0003-3891-5444

In [9]:
cmoriser.write()

CMORised output written to rsds_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_096110-096104.nc
